In [10]:
# Imports

from typing import Annotated, Any, Callable

from enum import StrEnum
from operator import itemgetter
# from pprint import pprint

from dotenv import dotenv_values

from pydantic.v1 import SecretStr
from tiktoken import encoding_for_model

from logging import basicConfig, DEBUG

from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.vectorstores import Qdrant
from langchain_community.document_loaders import PyMuPDFLoader

from langchain_core.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.output_parsers import StrOutputParser

from langchain_core.tools import BaseTool, tool # pyright: ignore [reportUnknownVariableType]
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, Runnable
from langchain_core.vectorstores import VectorStoreRetriever
from langchain_core.documents import Document
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI

from langgraph.prebuilt.chat_agent_executor import ToolExecutor, AgentState, create_react_agent # pyright: ignore [reportUnknownVariableType, reportMissingTypeStubs]
from langgraph.graph import END, StateGraph # pyright: ignore [reportMissingTypeStubs]

In [11]:
# Script

def clean_multi_line(text: str) -> str:
    return "\n".join(
        line.strip() for
        line in
        text
            .strip()
            .split("\n")
    )

def create_vectorstore(documents: list[Document]) -> VectorStoreRetriever:
    def chunk() -> list[Document]:
        gpt_3_4_turbo_encoding = encoding_for_model("gpt-3.5-turbo")

        def token_len(text: str) -> int:
            return len(gpt_3_4_turbo_encoding.encode(text))

        return RecursiveCharacterTextSplitter(
            chunk_size=300,
            chunk_overlap=0,
            length_function=token_len
        ).split_documents(
            documents
        )

    return Qdrant.from_documents(
        chunk(),
        OpenAIEmbeddings(model="text-embedding-3-small"),
        location=":memory:",
        collection_name="llama_3_paper"
    ).as_retriever()

def get_api_key(key_name: str) -> str:
    ENVIRONMENT_SECRETS = "environment_secrets"

    if not hasattr(get_api_key, ENVIRONMENT_SECRETS):
        setattr(get_api_key, ENVIRONMENT_SECRETS, dotenv_values())

    environment_secrets = getattr(get_api_key, ENVIRONMENT_SECRETS)

    api_key = environment_secrets[key_name]
    assert api_key != None
    return api_key

def create_agent_node[T](
    chat_model: ChatOpenAI,
    instructions: str,
    tools: ToolExecutor | list[BaseTool],
    state_schema: type[T]
):
    return create_react_agent(
        chat_model,
        tools
    )

def main():
    llama_3_paper_vectorstore = create_vectorstore(
        PyMuPDFLoader("https://arxiv.org/pdf/2404.19553").load()
    )

    rag_prompt = ChatPromptTemplate.from_template(
        clean_multi_line(
        """
        CONTEXT:
        {context}

        QUERY:
        {query}

        You are a helpful assistant. 
        Use the available context to answer the question. 
        If you can't answer the question, say you don't know.
        """
        )
    )

    chat_model = ChatOpenAI(
        model="gpt-3.5-turbo",
        api_key=SecretStr(get_api_key("OPENAI_API_KEY"))
    )

    get_query = itemgetter("query")

    rag_chain = (
        RunnableParallel(
            {"context": get_query | llama_3_paper_vectorstore, "query": get_query}
        ) |
        rag_prompt |
        chat_model |
        StrOutputParser()
    )

    return rag_chain.invoke(
        {"query": "What does 'context' refer to in 'long context'?"}
    )

In [12]:
# Run
output = None

if __name__ == "__main__":
    output = main()

output

"In the context of 'long context' as mentioned in the provided text, 'context' refers to a coherent text such as a book or a long paper. The term 'long context' implies that the text being referred to is extensive and requires analysis and aggregation of information from different locations within that text."

In [13]:
# Tests

def test1():
    print(
        clean_multi_line(
            """
            ArithmeticError
            Hello i am a multi
            line
            string
            that mean that i have many
            lines
            """
        )
    )